# Income Classification using TF method

Working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

### THE DATA

In [1]:
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
data=pd.read_csv('census_data.csv')

In [4]:
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [5]:
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

data['income_bracket']=data['income_bracket'].apply(label_fix)   


In [6]:
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [7]:
X=data.drop('income_bracket',axis=1)
y=data['income_bracket']

### Train Test Split on the Data

In [8]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [9]:
data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Import Tensorflow **

In [10]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **
** Create the continuous feature_columns for the continuous values using numeric_column **

In [11]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")
race = tf.feature_column.categorical_column_with_hash_bucket("race", hash_bucket_size=1000)


** Put all these variables into a single list with the variable name feat_cols **

In [12]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week,race]

### Create Input Function



In [13]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [14]:
embedded_education = tf.feature_column.embedding_column(education, dimension=16)

In [15]:
embedded_gender = tf.feature_column.embedding_column(gender, dimension=16)

In [16]:
embedded_marital_status = tf.feature_column.embedding_column(marital_status, dimension=16)

In [17]:
embedded_native_country = tf.feature_column.embedding_column(native_country, dimension=16)

In [18]:
embedded_occupation = tf.feature_column.embedding_column(occupation, dimension=16)

In [19]:
embedded_relationship= tf.feature_column.embedding_column(relationship, dimension=16)

In [20]:
embedded_workclass=tf.feature_column.embedding_column(workclass, dimension=16)
embedded_race=tf.feature_column.embedding_column(race, dimension=16)

In [21]:
feat_cols = [embedded_gender,embedded_occupation,embedded_marital_status,embedded_relationship,embedded_education,embedded_workclass,embedded_native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week,embedded_race]

In [22]:
dnn_model=tf.estimator.DNNClassifier(hidden_units=[13,13,13,13,13],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\raprabhu\\AppData\\Local\\Temp\\tmpizybqwt8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002CF817E5E10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
dnn_model.train(input_fn=input_func,steps=1000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\U

# Evaluate the model with test data

In [24]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [25]:
dnn_model.evaluate(input_fn=eval_input_func)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-23T13:54:39Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\raprabhu\AppData\Local\Temp\tmpizybqwt8\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-23-13:54:42
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.80264103, accuracy_baseline = 0.7582148, auc = 0.87696564, auc_precision_recall = 0.69390553, average_loss = 0.3769602, global_step = 1000, label/mean = 0.24178524, loss = 3.769216, precision = 0.7535047, prediction/mean = 0.26561618, recall = 0.27307367
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\raprabhu\AppData\L

{'accuracy': 0.80264103,
 'accuracy_baseline': 0.7582148,
 'auc': 0.87696564,
 'auc_precision_recall': 0.69390553,
 'average_loss': 0.3769602,
 'label/mean': 0.24178524,
 'loss': 3.769216,
 'precision': 0.7535047,
 'prediction/mean': 0.26561618,
 'recall': 0.27307367,
 'global_step': 1000}

# Accuracy using DNN # 80%

# Method # 2 using TF Linear Classifier

In [26]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week,race]

In [27]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\raprabhu\\AppData\\Local\\Temp\\tmp3e0xg364', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002CF8351DA20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [28]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\raprabhu\AppData\Local\Temp\tmp3e0xg364\model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 1
INFO:tensorflow:global_step/sec: 207.412
INFO:tensorflow:loss = 2.3047636, step = 101 (0.483 sec)
INFO:tensorflow:global_step/sec: 524.792
INFO:tensorflow:loss = 3.6252675, step = 201 (0.191 sec)
INFO:tensorflow:global_step/sec: 523.26
INFO:tensorflow:loss = 3.4921062, step = 301 (0.191 sec)
INFO:tensorflow:global_step/sec: 540.431
INFO:tensorflow:loss = 12.154338, step = 401 (0.185 sec)
INFO:tensorflow:global_step/sec: 510.148
INFO:tensorflow:loss = 177.01811, step = 501 (0.196 sec)
INFO:tensorflow:global_step/sec: 549.507
INFO:tensorflow:loss = 1.366975, step = 601 (0.184 sec)
INFO:tensorflow:global_step/

In [29]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [30]:
dnn_model.evaluate(input_fn=eval_input_func)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-23T13:54:57Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\raprabhu\AppData\Local\Temp\tmpizybqwt8\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-23-13:55:00
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.80264103, accuracy_baseline = 0.7582148, auc = 0.87696564, auc_precision_recall = 0.69390553, average_loss = 0.3769602, global_step = 1000, label/mean = 0.24178524, loss = 3.769216, precision = 0.7535047, prediction/mean = 0.26561618, recall = 0.27307367
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\raprabhu\AppData\Local\Temp\tmpizybqwt8\model.ckpt-1000


{'accuracy': 0.80264103,
 'accuracy_baseline': 0.7582148,
 'auc': 0.87696564,
 'auc_precision_recall': 0.69390553,
 'average_loss': 0.3769602,
 'label/mean': 0.24178524,
 'loss': 3.769216,
 'precision': 0.7535047,
 'prediction/mean': 0.26561618,
 'recall': 0.27307367,
 'global_step': 1000}

In [31]:
predictions = list(model.predict(input_fn=pred_fn))
predictions[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\raprabhu\AppData\Local\Temp\tmp3e0xg364\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'logits': array([-5.0538917], dtype=float32),
 'logistic': array([0.00634399], dtype=float32),
 'probabilities': array([0.99365604, 0.00634394], dtype=float32),
 'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object)}

In [32]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [33]:
final_preds

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,


In [34]:
from sklearn.metrics import classification_report

In [35]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.85      0.93      0.88      7407
           1       0.67      0.47      0.55      2362

   micro avg       0.82      0.82      0.82      9769
   macro avg       0.76      0.70      0.72      9769
weighted avg       0.80      0.82      0.80      9769



# Accuracy # 80%

# Both Methods giving 80% Accuracy

In [36]:

df = pd.DataFrame(list(zip(y_test,final_preds)), 
               columns =['Actual', 'Predicted']) 
df=pd.DataFrame(list(zip(X_test,y_test,final_preds)))

In [37]:
X_test.insert(loc=0, column='Actual', value=y_test)

In [38]:
X_test.insert(loc=0, column='Predicted', value=final_preds)

In [39]:
X_test

,Predicted,Actual,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
22278,0,0,27,Private,Some-college,10,Divorced,Adm-clerical,Unmarried,White,Female,0,0,44,United-States
8950,0,0,27,Private,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States
7838,0,0,25,Private,Assoc-acdm,12,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States
16505,0,0,46,Private,5th-6th,3,Married-civ-spouse,Transport-moving,Husband,Amer-Indian-Eskimo,Male,0,1902,40,United-States
19140,0,1,45,Private,11th,7,Divorced,Transport-moving,Not-in-family,White,Male,0,2824,76,United-States
12319,0,0,29,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Own-child,White,Male,0,0,75,United-States
28589,0,0,42,Local-gov,HS-grad,9,Separated,Other-service,Not-in-family,Black,Female,0,0,60,?
10000,0,0,34,Private,Some-college,10,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,50,United-States
28530,0,1,60,Local-gov,Assoc-voc,11,Divorced,Prof-specialty,Unmarried,White,Male,0,0,40,United-States
24237,0,0,19,Private,Some-college,10,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States


In [40]:
X_test.to_csv('to_compare.csv')